In [1]:
import sys
import os
sys.path.append("/home/hlife/Mamba-experiment/")

import torch
import torch.nn as nn
import torch.optim as optim
import argparse

from transformation_models.models import SimpleMLP
from dataset.hotpot_ssm_state import HotpotDoc1CacheIterator, HotpotDoc1PlusCacheIterator, extract_cache_single
from dataset.hotpot import HotpotQAIterator

from mamba.mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from transformers import AutoTokenizer


# Set up devices
mamba_device = f"cuda:6" if torch.cuda.is_available() else "cpu"
train_device = f"cuda:7" if torch.cuda.is_available() else "cpu"

mamba_model = MambaLMHeadModel.from_pretrained("state-spaces/mamba-2.8b", device=mamba_device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

/home/hlife/Mamba-experiment/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hlife/Mamba-experiment/transformation_models/../mamba/mamba_ssm/utils/hf.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializatio

In [2]:
dataset = HotpotQAIterator("/home/hlife/Mamba-experiment/dataset/HotpotQA/hotpot_train_v1.1.json").get_by_type("comparison")
print("Total comparison samples:", len(dataset))
print()
is_first = True
for item in dataset:
    if is_first:
        is_first = False
        print("id:", item.id)
        print("question:", item.question)
        print("answer:", item.answer)
        print("context number:", len(item.context))
        print("supporting fact number:", len(item.supporting_facts))
        print()
        
        useful_docs = item.get_useful()
        for i in range(len(useful_docs)):
            print(f"Document {i+1}: {useful_docs[i]['title']}\n{useful_docs[i]['content']}\n")
    if item.type != "comparison":
        assert False, "Data type error!"

Total comparison samples: 17456

id: 5a7a06935542990198eaf050
question: Which magazine was started first Arthur's Magazine or First for Women?
answer: Arthur's Magazine
context number: 10
supporting fact number: 2

Document 1: Arthur's Magazine
Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.  In May 1846 it was merged into "Godey's Lady's Book".

Document 2: First for Women
First for Women is a woman's magazine published by Bauer Media Group in the USA.  The magazine was started in 1989.  It is based in Englewood Cliffs, New Jersey.  In 2011 the circulation of the magazine was 1,310,696 copies.



In [3]:
# Get fixed doc 2 content and cache

fixed_doc2_id = "5a7a06935542990198eaf050"
hotpot_iterator = HotpotQAIterator("/home/hlife/Mamba-experiment/dataset/HotpotQA/hotpot_train_v1.1.json")
fixed_doc2_item = hotpot_iterator.get_by_id(fixed_doc2_id)

print("id:", fixed_doc2_item.id)
print("question:", fixed_doc2_item.question)
print("answer:", fixed_doc2_item.answer)
print("context number:", len(fixed_doc2_item.context))
print("supporting fact number:", len(fixed_doc2_item.supporting_facts))
print()

useful_docs = fixed_doc2_item.get_useful()
for i in range(len(useful_docs)):
    print(f"Document {i+1}: {useful_docs[i]['title']}\n{useful_docs[i]['content']}\n")

id: 5a7a06935542990198eaf050
question: Which magazine was started first Arthur's Magazine or First for Women?
answer: Arthur's Magazine
context number: 10
supporting fact number: 2

Document 1: Arthur's Magazine
Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.  In May 1846 it was merged into "Godey's Lady's Book".

Document 2: First for Women
First for Women is a woman's magazine published by Bauer Media Group in the USA.  The magazine was started in 1989.  It is based in Englewood Cliffs, New Jersey.  In 2011 the circulation of the magazine was 1,310,696 copies.



In [ ]:
from transformation_models.utils import generate_doc2_prompt, generate_doc12_prompt
doc2_prompt = generate_doc2_prompt(fixed_doc2_item.get_useful())
doc12_prompt = generate_doc12_prompt(fixed_doc2_item.get_useful())
print("Fixed doc2 prompt:", doc2_prompt)

from transformation_models.utils import extract_cache_single
cache_tensor = extract_cache_single(mamba_model, tokenizer, doc2_prompt, mamba_device)
print("Max in cache tensor:", torch.max(cache_tensor))
print("Min in cache tensor:", torch.min(cache_tensor))

cache_diff = extract_cache_single(mamba_model, tokenizer, doc12_prompt, mamba_device) - cache_tensor
print("Max in cache diff tensor:", torch.max(cache_diff))
print("Min in cache diff tensor:", torch.min(cache_diff))

Fixed doc2 prompt: Document 2: First for Women
First for Women is a woman's magazine published by Bauer Media Group in the USA.  The magazine was started in 1989.  It is based in Englewood Cliffs, New Jersey.  In 2011 the circulation of the magazine was 1,310,696 copies.


Max in cache tensor: tensor(4.8111, device='cuda:6')
Min in cache tensor: tensor(-7.6963, device='cuda:6')
Max in cache diff tensor: tensor(1.1550, device='cuda:6')
Min in cache diff tensor: tensor(-0.7551, device='cuda:6')
